In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

#print(os.environ.get('OPENAI_API_KEY'))

True

In [6]:
import json
from rich.console import Console
from rich.table import Table

def pretty_print(item):
    console = Console()
    with console.pager(styles=True):
        console.print(item)

In [8]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableMap

llm = OllamaLLM(base_url='http://localhost:11434', model="llama3.2:latest")

template = """
context: {context}
question: {question}
answer:
"""
prompt = ChatPromptTemplate.from_template(template)

def log(marker, log_data):
    pretty_print(f"Marker '{marker}':\n{log_data}\n")
    return log_data

qa_chain = (
    {
        "context": RunnablePassthrough(),
        "question": RunnablePassthrough(),
    }
    | prompt | (lambda x: log("prompt_log" , x))
    | llm
    | StrOutputParser()
)

context = "Robert bunked school on last Monday."
question = "When did Robert bunked school?"

result = qa_chain.invoke({"context": context, "question": question})
pretty_print(result)

Marker 'prompt_log':
messages=[HumanMessage(content="\ncontext: {'context': 'Robert bunked school on last Monday.', 'question': 'When 
did Robert bunked school?'}\nquestion: {'context': 'Robert bunked school on last Monday.', 'question': 'When did 
Robert bunked school?'}\nanswer:\n", additional_kwargs={}, response_metadata={})]



The answer to the question is: Last Monday.


In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnableMap, RunnablePassthrough

# Initialize the OpenAI client
llm = ChatOpenAI(
    model="gpt-4o-mini",
    #model="gpt-3.5-turbo",  # Specify the OpenAI model (e.g., "gpt-3.5-turbo" or "gpt-4")
    temperature=0.0
)

# Define the template
template = """
context: {context}
question: {question}
answer:
"""
prompt = ChatPromptTemplate.from_template(template)

# Define a logging function
def log(marker, log_data):
    print(f"Marker '{marker}':\n{log_data}\n")
    return log_data

# Create the QA chain
qa_chain = (
    RunnableMap(
        {
            "context": RunnablePassthrough(),
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | (lambda x: log("prompt_log", x))  # Log the intermediate prompt
    | llm
    | StrOutputParser()  # Parse the LLM output into a string
)

# Input data
context = "Robert bunked school last Monday."
question = "When did Robert bunk school?"

# Invoke the chain
result = qa_chain.invoke({"context": context, "question": question})

# Display the result
print("Final Result:\n", result)

Marker 'prompt_log':
messages=[HumanMessage(content="\ncontext: {'context': 'Robert bunked school last Monday.', 'question': 'When did Robert bunk school?'}\nquestion: {'context': 'Robert bunked school last Monday.', 'question': 'When did Robert bunk school?'}\nanswer:\n", additional_kwargs={}, response_metadata={})]

Final Result:
 Robert bunked school last Monday.


In [14]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message.content)

Function calls itself,  
Layers deep in logic's mind,  
Endless loop of thought.
